In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.sparse import hstack
import collections
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from xgboost import XGBClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pintusingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pintusingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/pintusingh/Desktop/fake-job-detection/venv/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B9202094-7D52-318C-99CF-7034B0E9F28D> /Users/pintusingh/Desktop/fake-job-detection/venv/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/

In [6]:
from scipy import sparse
import numpy as np
from scipy import stats
from scipy.sparse import vstack

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
data = pd.read_csv("../data/fake_job_postings.csv")

In [9]:
location = data["location"].str.split(",", expand= True, n= 2)
location.columns = ["country", "state", "city"]
data[["country", "state", "city"]] = location
data = data.drop(columns= "location")

In [10]:
salary = data["salary_range"].str.split("-", expand= True, n= 1)
data[["min_salary", "max_salary"]] = salary
data = data.drop(columns= "salary_range")

In [11]:
data = data.fillna("N/A")
data["state"] = data["state"].str.strip().apply(lambda x: "N/A" if x == '' else x)
data["country"] = data["country"].str.strip().apply(lambda x: "N/A" if x == '' else x)
data["city"] = data["city"].str.strip().apply(lambda x: "N/A" if x == '' else x)

In [12]:
for i in ["company_profile", "description", "requirements", "benefits"]:
    data[i] = data[i].str.lower()

In [13]:
data.isnull().sum()

job_id                 0
title                  0
department             0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
country                0
state                  0
city                   0
min_salary             0
max_salary             0
dtype: int64

Prototype Selection with K-Means + TFIDF

In [14]:
# tokenizer define
ps = PorterStemmer()
stop = set(stopwords.words('english'))
def tokenizer (doc):
    sentences = sent_tokenize(doc)
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        words = [ps.stem(word) for word in words]
        tokens+=words
    return [w.lower() for w in tokens if w not in stop]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_selection import RFE
from matplotlib import pyplot


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pintusingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pintusingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine text features
df1 = data.copy()
text_feature = df1[['title', 'department','company_profile','description','requirements','benefits']].apply(lambda x: ' '.join(x), axis = 1)

# TF-IDF without custom tokenizer (much faster)
tfidf = TfidfVectorizer(
    stop_words='english',   # automatically removes stopwords
    max_features=5000,      # optional: limit number of features to speed up
    lowercase=True,
    use_idf=True,
    norm='l2',
    smooth_idf=True
)

text_feature = tfidf.fit_transform(text_feature)
print("TF-IDF shape:", text_feature.shape)

TF-IDF shape: (17880, 5000)


In [21]:
lb = LabelEncoder()

for col in ['employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country',
       'state', 'city']:
    df1[col] = lb.fit_transform(df1[col])

In [22]:
# scale
label_feature = df1[['employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country',
       'state', 'city']]
scaler = StandardScaler().fit(label_feature)

label_feature = scaler.transform(label_feature)

In [23]:
X = hstack((text_feature, label_feature))
y = df1['fraudulent']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10, random_state= 42, stratify= y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.11, random_state= 42, stratify= y_train)

In [25]:
kmx = X_train[np.array(y_train == 0), :]

In [26]:
y_train.value_counts()

fraudulent
0    13628
1      693
Name: count, dtype: int64

In [27]:
#Use K-means to select 693 non-fraudulent cases
km = KMeans(n_clusters= 693, random_state= 42).fit(kmx)
non_fraud_centers = km.cluster_centers_

In [28]:
X_train_ps = vstack([sparse.csr_matrix(non_fraud_centers), X_train[np.array(y_train == 1), :]])
y_train_ps = np.concatenate([np.repeat(0, 693), np.repeat(1, 693)])

In [29]:
#KNN
knn = KNeighborsClassifier(n_neighbors= 1)
knn.fit(X_train_ps, y_train_ps)
y_val_pred = knn.predict(X_val)
print(classification_report(y_val, y_val_pred, digits= 6))

              precision    recall  f1-score   support

           0   0.981054  0.983383  0.982217      1685
           1   0.658537  0.627907  0.642857        86

    accuracy                       0.966121      1771
   macro avg   0.819795  0.805645  0.812537      1771
weighted avg   0.965392  0.966121  0.965738      1771



In [30]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_pred)
metrics.auc(fpr, tpr)

0.8056448830308467

In [ ]:
y_test_pred_2 = knn.predict(X_test)
print(classification_report(y_test, y_test_pred_2, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_pred_2)
metrics.auc(fpr, tpr)

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors= 1)
knn2.fit(X_train, y_train)
y_val_pred_2 = knn2.predict(X_val)
print(classification_report(y_val, y_val_pred_2, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_pred_2)
metrics.auc(fpr, tpr)

In [ ]:
y_val_test_2 = knn2.predict(X_test)
print(classification_report(y_test, y_val_test_2, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_val_test_2)
metrics.auc(fpr, tpr)

Prototype Selection with K-Means + Countvectorizer

In [31]:
# combine text features and vectorize
df2 = data.copy()
text_feature = df2[['title', 'department','company_profile','description','requirements','benefits']].apply(lambda x: ' '.join(x), axis = 1)

bow = CountVectorizer(tokenizer=tokenizer)

text_feature = bow.fit_transform(text_feature)

/Users/pintusingh/Desktop/fake-job-detection/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
# encode label features
lb = LabelEncoder()

for col in ['employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country',
       'state', 'city']:
    df2[col] = lb.fit_transform(df2[col])


In [ ]:
# scale
label_feature = df2[['employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country',
       'state', 'city']]
scaler = StandardScaler().fit(label_feature)

label_feature = scaler.transform(label_feature)

In [ ]:
X = hstack((text_feature, label_feature))
y = df1['fraudulent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10, random_state= 42, stratify= y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.11, random_state= 42, stratify= y_train)
kmx = X_train[np.array(y_train == 0), :]

In [ ]:
y_train.value_counts()

In [ ]:
#Use K-means to select 693 non-fraudulent cases
km = KMeans(n_clusters= 693, random_state= 42).fit(kmx)
non_fraud_centers = km.cluster_centers_

In [ ]:
X_train_ps = vstack([sparse.csr_matrix(non_fraud_centers), X_train[np.array(y_train == 1), :]])
y_train_ps = np.concatenate([np.repeat(0, 693), np.repeat(1, 693)])

In [ ]:
#KNN
knn = KNeighborsClassifier(n_neighbors= 1)
knn.fit(X_train_ps, y_train_ps)
y_val_pred = knn.predict(X_val)
print(classification_report(y_val, y_val_pred))

In [ ]:
y_val_pred = knn.predict(X_val)
print(classification_report(y_val, y_val_pred, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_pred)
metrics.auc(fpr, tpr)

In [ ]:
y_test_pred = knn.predict(X_test)
print(classification_report(y_test, y_test_pred, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_pred)
metrics.auc(fpr, tpr)

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors= 1)
knn2.fit(X_train, y_train)
y_val_pred_2 = knn2.predict(X_val)
print(classification_report(y_val, y_val_pred_2))

In [ ]:
print(classification_report(y_val, y_val_pred_2, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_pred_2)
metrics.auc(fpr, tpr)

In [ ]:
y_test_pred_2 = knn2.predict(X_test)
print(classification_report(y_test, y_test_pred_2, digits= 6))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_pred_2)
metrics.auc(fpr, tpr)